In [1]:
import sys, os

import pandas as pd
import geopandas as gpd

In [2]:
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_lines  = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polylines_CntryandDisp.shp"

inB = gpd.read_file(global_bounds)
inL = gpd.read_file(global_lines)

In [3]:
bad_shapes = inL['geometry'].apply(lambda x: x is None)
inL = inL.loc[~bad_shapes]
#all_borders = inL.unary_union

In [4]:
trade_file = "/home/wb411133/temp/tradecostsforactualborders.csv"
trade_data = pd.read_csv(trade_file)
trade_data.head()

,reporter,partner,neighbors,trade_cost
0,AFG,CHN,True,293.428099
1,AFG,IRN,True,121.637338
2,AFG,PAK,True,94.850399
3,AFG,TJK,True,171.974544
4,AFG,UZB,True,260.956700


In [ ]:
inB['geometry'] = inB['geometry'].apply(lambda x: x.buffer(100))

In [6]:
trade_data.head()

,reporter,partner,neighbors,trade_cost
0,AFG,CHN,True,293.428099
1,AFG,IRN,True,121.637338
2,AFG,PAK,True,94.850399
3,AFG,TJK,True,171.974544
4,AFG,UZB,True,260.956700


In [7]:
res = []
for idx, row in trade_data.iterrows():
    print(f"{idx} of {trade_data.shape[0]}")
    #Select country bounds
    cur_country = inB.loc[inB['ISO3'] == row['reporter']]
    country_shape = cur_country.unary_union.buffer(100)
    #select partner bounds
    partner_country = inB.loc[inB['ISO3'] == row['partner']]
    partner_shape = partner_country.unary_union.buffer(100)
    # Only continue if the reporter and partner are the same
    if (country_shape.intersects(partner_shape)):
        #Dissolve national bounds intersecting from country 
        national_lines = inL.loc[inL['geometry'].apply(lambda x: x.intersects(country_shape))]
        national_lines = national_lines.unary_union.intersection(country_shape)
        national_lines = national_lines.intersection(partner_shape)
        res.append([row['reporter'], row['partner'], row['trade_cost'], national_lines])

0 of 446
1 of 446
2 of 446
3 of 446
4 of 446
5 of 446
6 of 446
7 of 446
8 of 446
9 of 446
10 of 446
11 of 446
12 of 446
13 of 446
14 of 446
15 of 446
16 of 446
17 of 446
18 of 446
19 of 446
20 of 446
21 of 446
22 of 446
23 of 446
24 of 446
25 of 446
26 of 446
27 of 446
28 of 446
29 of 446
30 of 446
31 of 446
32 of 446
33 of 446
34 of 446
35 of 446
36 of 446
37 of 446
38 of 446
39 of 446
40 of 446
41 of 446
42 of 446
43 of 446
44 of 446
45 of 446
46 of 446
47 of 446
48 of 446
49 of 446
50 of 446
51 of 446
52 of 446
53 of 446
54 of 446
55 of 446
56 of 446
57 of 446
58 of 446
59 of 446
60 of 446
61 of 446
62 of 446
63 of 446
64 of 446
65 of 446
66 of 446
67 of 446
68 of 446
69 of 446
70 of 446
71 of 446
72 of 446
73 of 446
74 of 446
75 of 446
76 of 446
77 of 446
78 of 446
79 of 446
80 of 446
81 of 446
82 of 446
83 of 446
84 of 446
85 of 446
86 of 446
87 of 446
88 of 446
89 of 446
90 of 446
91 of 446
92 of 446
93 of 446
94 of 446
95 of 446
96 of 446
97 of 446
98 of 446
99 of 446
100 of 446

In [8]:
xx = gpd.GeoDataFrame(res, columns=['FROM', 'TO', 'VAL', 'geometry'], crs=inB.crs)
xx.shape

(418, 4)

In [9]:
xx.to_file("/home/wb411133/temp/tradecostsforactualborders.shp")

In [ ]:
xx

In [ ]:
xx.unary_union